In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from IPython.html import widgets
from IPython.html.widgets import interact

:0: FutureWarning: IPython widgets are experimental and may change in the future.


# Fourier Series Demo - Tutorial format

This notebook is part of the series of postings on iPython's Rich Display System in the [All Things Computing blog](http://fxmartins.com). This is the "tutorial version", intended for those following the code development step-by-step. If you just want to use the end result of the code in your notebooks, see the [production version](file:../Production/.

## Step 1

The `interact()` function creates an interactive widget in the notebook and calls a user-defined function (the _callback_ function) whenever there are changes in the interface. The callback is designed to reflect the changes in the interface. The following cell define a callback that simply prints a message reporting what is the value of `n`.

In [2]:
def plot_fourier_series(**kwargs):
    n = kwargs['n']
    print 'The value of n is: {:d}'.format(n)

The callback must have a single argument, of the form `**kwargs`, that is, it can only accept keyword arguments. The dictionary `kwargs` will be filled by the call to `interact()`, which is demonstrated below:

In [3]:
_i = interact(plot_fourier_series,
              n=(1, 20))

The value of n is: 10


Now, each time the value of `n` is changed with the slide, the output line at the botton is automatically updated.

The first argument to `interact()` is the callback, and all other arguments are widget specifications. There are two ways of defining the widgets: using an _abbreviation_ (as in the example above), or calling a widget constructor (see example below). I personally prefer to always use a constructor, since it allows more flexibility. The available abbreviations are:
<table style="background-color:#FFFAF0;">
    <tr>
        <th> Abbreviation </th>
        <th> Description </th>
        <th> Widget Class</th>
    </tr>
    <tr>
        <td> _key=(start, end, step)_, where _start_, _end_ and _step_ are integers</td>
        <td> An integer slider widget </td>
        <td>  IntSlideWidget </td>
    </tr>
    <tr>
        <td> _key=(start, end, step)_, where _start_, _end_ and _step_ are floats</td>
        <td> A float slider widget. </td>
        <td>  FloatSlideWidget </td>
    </tr>
    <tr>
        <td> _key='string'_</td>
        <td> A text box that can be edited by the user </td>
        <td> TextWidget </td>
    </tr>
    <tr>
        <td> _key=True_ or _key=False_</td> 
        <td> A check box </td>
        <td> CheckboxWidget </td>
    </tr>
    <tr>
        <td> _key=['List', 'of', 'strings']_</td>
        <td> A selection box</td>
        <td> SelectWidget </td>
    </tr>
</table>

Each time a widget in the interface changes, `interact()` creates a dictionary with a key/value pair corresponding to each widgets. Then, it calls the callback, passing the dictionary as ``**kwargs``.

(Note: we assign the return value of `interact()` to the dummy variable `_i` to prevent an annoying text output.)

## Step 2

Using abbreviations to define widgets has some severe limitations: the widget key must be a valid Python identifier; not all kinds of widgets are available as abbreviations; and the layout of the widgets cannot be changed.

In our example, the variable `n` is used to represent the number of terms in the Fourier series approximation. We would like to use a more descriptive name for the integer slide widget controlling `n`.

We first change the callback in the following way:

In [28]:
def plot_fourier_series(**kwargs):
    n = kwargs['Number of terms:']
    print 'The number of terms is: {:d}'.format(n)

The key corresponding to a widget in the dictionary `kwargs` is set in the `description` property of the widget. This is how the call to `interact()` is modified:

In [29]:
_i = interact(plot_fourier_series,
              n=widgets.IntSliderWidget(description='Number of terms:',
                                        min=1, max=20, value=5))

KeyError: 'Number of terms:'

Notice the call to the constructor `IntSliderWidget`. Besides setting the `description` property, we also set the properties characterizing the minimum, maximum and initial value of the slider. The available widgets and properties will be dealt with in a future tutorial.

## Step 3

We are now ready to create the first version of the `interact()` that does something interesting. All we need to do is change the callback so that a graph of the Fourier series is produced:

In [ ]:
def plot_fourier_series(**kwargs):
    n = kwargs['Number of terms:']
    pi = np.pi
    xbound = pi
    ybound = 1.5
    plt.figure(figsize=(7,5))
    plt.subplot(1, 1, 1, axisbg='FloralWhite')
    plt.axis([-xbound, xbound, -ybound, ybound])
    x = np.linspace(-xbound, xbound, 300)
    f = np.piecewise(x, [x < 0, x >= 0], [-1, 1])
    fs_terms = (4.0 / pi) * np.array([np.sin((2 * k + 1) * x) / (2 * k + 1) 
                                      for k in range(n)])
    fs = np.sum(fs_terms,0)
    plt.plot(x, f, lw = 2, color='Green')
    plt.plot(x, fs, lw = 2.5, color='DarkRed', alpha=0.75)
    plt.axhline(color='Gray')
    plt.title('Fourier Series of a square wave, {:d} terms'.format(n))

In this version, the callback computes the Fourier series for a square wave. Most of the code consists of calls to `matplotlib.pyplot` functions (imported as `plt`). The square wave and the Fourier approximation are computed in the lines:

    x = np.linspace(-xbound, xbound, 300)
    f = np.piecewise(x, [x < 0, x >= 0], [-1, 1])
    fs_terms = (4.0 / pi) * np.array([np.sin((2 * k + 1) * x) / (2 * k + 1) 
                                      for k in range(n)])
    fs = np.sum(fs_terms,0)

This is a straightforward `numpy` implementation of the formula:
$$
\frac{4}{\pi}\sum_{k=0}^{\infty}\frac{\sin((2k+1)x)}{2k+1},
$$
truncated to the first _n_. There are no changes at all to the `interact()` call:

In [ ]:
_i = interact(plot_fourier_series,
               n=widgets.IntSliderWidget(description='n',
                                         min=1, max=20, value=5))

## Step 4

We now want to change the demo to include two more functions: a triangular wave, and a sawtooth wave. The code of the callback becomes somewhat more complex:

In [ ]:
def plot_fourier_series(**kwargs):
    n = kwargs['Number of terms:']
    wave_type = kwargs['Function:']
    pi = np.pi
    xbound = pi
    ybound = 1.75
    plt.figure(figsize=(7,5))
    plt.subplot(1, 1, 1, axisbg='FloralWhite')
    plt.axis([-xbound, xbound, -ybound, ybound])
    x = np.linspace(-xbound, xbound, 300)
    if wave_type == 'Square wave':
        f = np.piecewise(x, [x < 0, x >= 0], [-1, 1])
        fs_terms = (4.0/pi) * np.array([np.sin((2*k+1)*x) / (2*k+1) 
                                        for k in range(n)])
    elif wave_type == 'Triangular wave':
        f = (2.0/pi) * np.piecewise(x, [x < -pi/2, (-pi/2 <= x) & (x < pi/2), x >= pi/2],
                         [lambda u: -pi-u, lambda u: u, lambda u: pi-u])
        fs_terms = (8/pi**2) * np.array([(-1)**k * np.sin((2*k+1)*x) / (2*k+1)**2 
                                         for k in range(n)])
    else:
        f = np.piecewise(x, [x < 0, x >=0], 
                         [lambda u: -1-u/pi, lambda u:1-u/pi])
        fs_terms = (2/pi) * np.array([np.sin(k*x) / k 
                                      for k in range(1,n+1)])
    fs =  np.sum(fs_terms,0)
    plt.plot(x, f, lw = 2, color='Green')
    plt.plot(x, fs, lw = 2.5, color='DarkRed', alpha=0.75)
    plt.axhline(color='Gray')
    plt.title('Fourier Series of a {:s}, {:d} terms'.format(wave_type.lower(),n))

The formulas we are using to compute the Fourier series are:
$$
\text{Triangular wave:}\quad \frac{8}{\pi^2} \sum_{k=0}^{\infty} \frac{(-1)^k\sin((2k+1)x)}{(2k+1)^2}
$$
$$
\text{Sawtooth wave:}\quad \frac{2}{\pi} \sum_{k=1}^{\infty} \frac{\sin(kx)}{k}
$$
The call to interact now contains the specification of a `RadioButtonWidget`:

In [ ]:
_i = interact(plot_fourier_series,
              n=widgets.IntSliderWidget(description='n',
                                        min=1, max=20, value=5),
              b=widgets.RadioButtonsWidget(description='Function',
                                           values=['Square wave',
                                                   'Triangular wave',
                                                   'Sawtooth wave']))

Notice the curious fact that the order in which the widgets appear is not the same as they were specified in `interact()`. If you understand how Python function calls work, you know what is going on: the keyword arguments in a function call are passed in a dictionary, and dictionaries in Python are not ordered.

In the next tutorial we will cover the full range of widgets, and see how do more precise layouts.